In [1]:
import os
os.chdir('../')

In [2]:
from models.GCNmodel import *
from utils import *
import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchsummary import summary
import time
import numpy as np
import pandas as pd

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size=1024
dataset=elementtables_structure_new(search=True)
task='stability'
model_stable=attention_CNN_quadtask().to(device)
model_stable.load_state_dict(torch.load('result/lattice_constant_E0_Ef_stability_model.pth'))
dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=False)
threshold=0.1
material_name=[]
mag=[]
a=[]
data_stable=[]
spacegroup=[]
stability=[]
model_stable.eval()
with torch.no_grad():
    for data in dataloader:
        out=calculate_result(data,model_stable,task)
        for i,stability_i in enumerate(out):
            if stability_i<threshold:
                name=ChemicalSymbols[torch.argwhere(data[0][i,-4:,:,:].reshape(4,100)!=0)[:,1].to('cpu').numpy()]
                if len(name)==3:
                    name=np.insert(name,torch.argwhere(torch.sum(data[0][i,-4:,:,:].reshape(4,100),dim=1)==0)[0].to('cpu').numpy()[0],'X')
                material_name.append(name)
                data_stable.append(data[0][i])
                mag.append(data[1][i])
                spacegroup.append(data[2][i])
                a.append(data[3][i])
                stability.append(stability_i)
            else:
                continue

In [4]:
task="Mag"
model_mag=attention_CNN(end_with_activation=True).to(device)
model_mag.load_state_dict(torch.load('result/Mag_model.pth'))
mag_pred=calculate_result([torch.stack(data_stable),[],torch.stack(spacegroup).reshape(-1),torch.stack(a).reshape(-1,1)],model_mag,task)

In [5]:
torch.stack(data_stable).shape

torch.Size([1267, 8, 10, 10])

In [6]:
max(mag),min(mag)

(tensor([2.1151], device='cuda:0'), tensor([0.0101], device='cuda:0'))

In [7]:
order_list_with_index=sorted(enumerate(mag),key=lambda x:x[1],reverse=True)
ordered_mag=[x[1] for x in order_list_with_index]
ordered_material=[material_name[x[0]] for x in order_list_with_index]
ordered_a=[a[x[0]] for x in order_list_with_index]
ordered_mag_pred=[mag_pred[x[0]] for x in order_list_with_index]
ordered_stability=[stability[x[0]] for x in order_list_with_index]

In [8]:
ordered_a[0].to('cpu').numpy()[0]

4.2651005

In [9]:
for i in range(40):
    print('[',end='')
    for j in range(4):
        if ordered_material[i][j]=='X':
            continue
        print(ordered_material[i][j],end=', ' if j!=3 else '')
    print(']/',end=' ')
    print(ordered_mag[i].to('cpu').numpy()[0],end=' ')
    print(ordered_mag_pred[i].detach().to('cpu').numpy()[0],end=' ')
    print(ordered_stability[i].detach().to('cpu').numpy()[0])

[Ir, Fe, Rh, Fe]/ 2.115125 2.0391192 0.07649259
[In, Au, Gd, Au]/ 1.80374 1.496893 0.0025981069
[Ba, Ca, In]/ 1.6595134 1.7972585 0.029836833
[In, Au, Sm, Au]/ 1.4356275 2.114887 0.007589191
[In, Ag, Sm, Ag]/ 1.4201825 2.152902 0.081501156
[Fe, Co, Ge, Co]/ 1.4114175 1.533891 0.024230659
[Fe, Co, Sn, Co]/ 1.4028875 1.6428018 0.08085045
[Fe, Co, Ti, Co]/ 1.32944 1.5026723 0.051258475
[Fe, Co, Ga, Co]/ 1.25114 1.4337418 -0.00023772358
[Ga, Co, Fe, Co]/ 1.2511375 1.4304601 -0.00023884088
[Al, Co, Fe, Co]/ 1.237475 1.3858738 0.03860441
[Rh, Ga, Co, Fe]/ 1.2319275 1.2200052 0.03896895
[Ir, Ga, Co, Fe]/ 1.221125 1.2797523 0.0675475
[Pt, Be, Co, Fe]/ 1.219435 1.2273743 0.097671
[Mg, Co, Fe, Ir]/ 1.165335 1.3072871 0.08635618
[Fe, Ru, Nb, Ru]/ 1.156235 1.4487209 0.07223901
[Fe, Ru, Ta, Ru]/ 1.152515 1.2670466 0.028865427
[Co, Fe, Al, Co]/ 1.14517 1.2134086 0.091312736
[La, La, Ca, In]/ 1.14303 1.8683939 0.08958644
[Be, Co, Fe, Co]/ 1.1367176 1.2515326 0.08034104
[Fe, Ru, V, Ru]/ 1.12835 1.3156

In [11]:
name_wo_X=[]
for i in range(20):
    name_wo_X.append([x for x in ordered_material[i] if x!='X'])

for i in range(20):
    with open('search/contrast/meta_stable/%s.vasp'%(''.join(name_wo_X[i])),'w') as f:
        f.write('elements: %s\n'%(''.join(name_wo_X[i])))
        f.write('1.000\n')
        f.write('     %f    %f    %f\n'%(0,ordered_a[i].to('cpu').numpy()[0]/(2**0.5),ordered_a[i].to('cpu').numpy()[0]/(2**0.5)))
        f.write('     %f    %f    %f\n'%(ordered_a[i].to('cpu').numpy()[0]/(2**0.5),0,ordered_a[i].to('cpu').numpy()[0]/(2**0.5)))
        f.write('     %f    %f    %f\n'%(ordered_a[i].to('cpu').numpy()[0]/(2**0.5),ordered_a[i].to('cpu').numpy()[0]/(2**0.5),0))
        element_set=set(name_wo_X[i])
        for element in element_set:
            f.write('   %s'%element)
        f.write('\n')
        for element in element_set:
            f.write('   %d'%list(name_wo_X[i]).count(element))
        f.write('\nDirect\n')
        for element in element_set:
            args=np.argwhere(np.array(name_wo_X[i])==element).reshape(-1)
            for arg in args:
                f.write('   %f    %f    %f\n'%(arg*0.25,arg*0.25,arg*0.25))